## Import Libraries

In [1]:
from os import path, makedirs
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## Power Decoding - Partial//Detailed Encoding
### For ALL time points

In [34]:
def def_freqname(freq):
    freqnames = ['0_theta', '1_alpha', '2_beta', '3_gamma']
    freqname = freqnames[freq]
    return freqname

def odor_su_dict(phase):
    if phase == 'Encoding':
        odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
                'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
                'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
                'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
                'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
                'MICP': {2:6,12:8,6:96,8:8,3:12,18:4,9:6,14:10},
                'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
    else:
        odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
                'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
                'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68},#15:4 #remove for TPSim 15:4
                'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
                'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
                'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
    return odors_su

def norm_power_by_roi(phase,su,roi,freq,val):
    if val == 'xpow':
        path_pow = path.join(st.path, 'feature/0_Power_'+phase+'_By_Odor/')
        mat_file = path_pow+'{}_odor_{}_bipo_sel_physFT_pow.npz'
    elif val == 'tps':
        path_pow = path.join(st.path, 'feature/TPSim_Enc_Ret_By_Odor_all/TPS_R_p_by_odor/')
        mat_file = path.join(path_pow, 'TPS_spear_{}_odor_{}_{}.npz')
        
    all_odor_pow, scores = np.array([]), []
    odors_su = odor_su_dict(phase)
    fname = def_freqname(freq)
    for odor in odors_su[su]:
        if val == 'tps':
            mat = np.load(mat_file.format(su,odor,fname))
            xpow, Mai_RL, channels = mat[val], mat['label'], mat['channel']
        else:
            mat = np.load(mat_file.format(su,odor))
            xpow, Mai_RL, channels = mat[val][freq,...], mat['Mai_RL'], mat['channels']      
        if roi == 'pPirT':
            id_rois = [r in ['pPirT','Amg','Amg-PirT'] for r in Mai_RL]
        else:
            id_rois = [r == roi for r in Mai_RL]
        xpow = np.mean(xpow[id_rois,27:47,:],axis=-2) if val == 'xpow' else xpow[id_rois,...]
        #xpow_b = np.mean(xpow[id_rois,22:27,:],axis=-2)
        #xpow = (xpow_a - xpow_b) / xpow_b#
        score_roi = [odors_su[su][odor]]*xpow.shape[-1]
        scores.append(score_roi)
        all_odor_pow = np.concatenate((all_odor_pow,xpow),axis=-1) if np.size(all_odor_pow) else xpow
    scores = np.concatenate(scores,axis=0)
    #score_su_norm = (scores - scores.mean())/scores.std()
    #mean_roi, std_roi = np.mean(np.ravel(all_odor_pow)), np.std(np.ravel(all_odor_pow))
    #all_odor_pow = (all_odor_pow - mean_roi)/ std_roi
    kwargs = {}
    kwargs['label'], kwargs['channels']= Mai_RL[id_rois], channels[id_rois]
    kwargs['xyz'], kwargs[val] = mat['xyz'][id_rois], all_odor_pow
    kwargs['fname'], kwargs['nelecs'] = fname, all_odor_pow.shape[0]
    kwargs['scores'] = np.array(scores)
    return kwargs

def groups_pow_scores(phase,su,roi,freq,val):
    dict_ = norm_power_by_roi(phase,su,roi,freq,val)
    scores = dict_['scores']
    pow_norm = dict_[val]
    #print(len(scores),pow_norm.shape)
    #steps = np.percentile(scores,[0,25,50,75,100])
    steps = np.percentile(scores,[0,100])
    pow_groups = []
    if pow_norm.shape[0] == 1:
        for g,step in enumerate(steps[:-1]):
            pow_groups.append(pow_norm[:,np.where((step<=scores)&(scores<=(steps[g+1])))[0]])
    else:
        for g,step in enumerate(steps[:-1]):
            pow_groups.append(np.squeeze(pow_norm[:,np.where((step<=scores)&(scores<=(steps[g+1])))]))
            
    print('groups shapes',[pow.shape for pow in pow_groups])
    labels, weights = [], []
    for i,_ in enumerate(pow_groups):
        labels.append([i]*pow_groups[i].shape[1])
        weights.append([pow_groups[i].shape[1]])
    labels = np.concatenate(labels,axis=0)
    weights = np.concatenate(weights,axis=0)
    return pow_groups, labels, weights, dict_

In [36]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.metrics import roc_auc_score, accuracy_score
from numpy.random import permutation
from scipy import stats
from matplotlib.offsetbox import AnchoredText
from joblib import delayed, Parallel

st = study('Olfacto')
save_path = path.join(st.path, 'classified/_multi_classes/LDA_Power_E_2classes_BBG_no_norm_k5_auc_all/')
subjects = ['LEFC','MICP','VACJ','SEMC','FERJ','PIRJ','CHAF']
rois = ['ACC','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT']
phase = 'Encoding'
freqs = 4
nperm = 1000
val = 'xpow'

def classif_n_classes_by_subj(su):
    for freq in range(freqs):
        for roi in rois:
            pow_groups, labels, weights, dict_ = groups_pow_scores(phase,su,roi,freq,val)
            freq_name, nelecs = dict_['fname'], dict_['nelecs']
            if nelecs > 0:
                for elec_num in range(nelecs): 
                    elec, elec_label = dict_['channels'][elec_num], dict_['label'][elec_num]
                    xyz = dict_['xyz'][elec_num]
#                     print ('elec ', elec, 'elec_label ', elec_label)

                    #Filenames to save
                    name_auc = (save_path+freq_name+'/auc/'+su +'_auc_3classes_'+roi+'_('+str(elec_num)+').npy')
                    name_perm = (save_path+freq_name+'/auc/'+su +'_perm_3classes_'+roi+'_('+str(elec_num)+').npy')
                    plot_name = (save_path+freq_name+'/fig/'+su +'_Power_3classes_'+roi+'_('+str(elec_num)+').png')    
                    
                    if path.exists(name_auc):
                        print(su,phase,elec_num,freq,'already computed')
                    else:
                        print('--» processing',roi, su, 'elec', elec_num,'/',nelecs, 'freq',freq)

                        pow_data_elec = []
                        for i,power in enumerate(pow_groups):
                            pow_data_elec.append(power[elec_num])
                # =============================  Classification Computation ============================================================           
                        # create a data matrix, concatenate along the trial dimension
                        x = np.concatenate(pow_data_elec, axis=0)[:,np.newaxis]
                        print ('Size of the concatenated data: ', x.shape)
                        y = labels
                        print ('Size of label for classif: ', len(y))

                        auc = np.array([])
                        for t in range(x.shape[1]):
                            X = x[:,t]
                            X = X.reshape(-1, 1)
                            score_rep = []
                            for i in range(5):
                                k = 5
                                skf = SKFold(n_splits=k, random_state=None, shuffle=True)
                                skf.get_n_splits(X, y)
                                score_cv = []
                                for train_index, test_index in skf.split(X, y):
                                    clf = LDA()
                                    X_train, X_test = X[train_index], X[test_index]
                                    y_train, y_test = y[train_index], y[test_index]
                                    clf.fit(X=X_train, y=y_train)
                                    y_pred = clf.predict(X_test)
                                    score_cv.append(roc_auc_score(y_test,y_pred))
                                score_rep.append(np.mean(score_cv))
                            score_rep = np.asarray(score_rep).reshape(1,len(score_rep))
                            auc = np.vstack((auc, score_rep)) if np.size(auc) else score_rep
                        auc = np.swapaxes(auc,0,1)
                        DA = np.mean(auc)

                        perm_scores = np.array([])
                        for t in range(x.shape[1]):
                            X = x[:,t]
                            X = X.reshape(-1, 1)
                            perm_rep = []
                            for perm in range(nperm):
                                y_perm = y[permutation(len(y))]
                                score_cv = []
                                for train_index, test_index in skf.split(X, y_perm):
                                    clf = LDA()
                                    X_train, X_test = X[train_index], X[test_index]
                                    y_train, y_test = y_perm[train_index], y_perm[test_index]
                                    clf.fit(X=X_train, y=y_train)
                                    y_pred = clf.predict(X_test)
                                    score_cv.append(roc_auc_score(y_test,y_pred))
                                perm_rep.append(np.mean(score_cv))
                            perm_rep = np.asarray(perm_rep).reshape(1,len(perm_rep))
                            perm_scores = np.vstack((perm_scores, perm_rep)) if np.size(perm_scores) else perm_rep
                        perm_scores = np.swapaxes(perm_scores,0,1)           
                        th_0_05_perm = perm_pvalue2level(perm_scores, p=0.05, maxst=True)
                        th_0_01_perm = perm_pvalue2level(perm_scores, p=0.01, maxst=True)
                        print('th_perm 005: ', th_0_05_perm[0], '001',th_0_01_perm[0], 
                              'auc_max', np.max(auc), 'auc_mean', np.mean(auc))

                # ============================== PLOT POWER ANALYSIS + STATS & DECODING ACCURACY ===================================================
                        #if DA >= th_0_05_perm[0]:
                        # plot and figure parameters
                        xfmt = ScalarFormatter(useMathText=True)
                        xfmt.set_powerlimits((0,3))
                        fig, ax = plt.subplots(figsize=(7,7))
                        title = freq_name+' for '+su+' 4 classes '+str(elec)+' '+str(elec_label)+' coord '+str(xyz)
                        fig.suptitle(title, fontsize=12)

                        # Plot the POW + STATS
                        plt.xlabel('EpiScore'), plt.ylabel(freq_name)
                        anchored_text = AnchoredText('DA = %s, th = %s' % (np.round(DA,2), round(th_0_05_perm[0],2)), loc=2)
                        ax.add_artist(anchored_text)
                        xticks, w = np.arange(0,2), 0.8
                        means = [pow.mean()for pow in pow_data_elec]
                        stds = [stats.sem(pow) for pow in pow_data_elec]
                        plt.bar(xticks,means,color='blue',yerr=stds)
                        plt.savefig(plot_name, dpi=300, bbox_inches='tight')
                        plt.clf()
                        plt.close()

                        #Save plots
                        np.save(name_auc, auc)
                        np.save(name_perm, perm_scores)
                        del X, auc, pow_data_elec
            del pow_groups

if __name__ == "__main__":
    Parallel(n_jobs=-1)(delayed(classif_n_classes_by_subj)(su) for su in subjects)
    #classif_4_classes_by_subj('LEFC') 

-> Olfacto loaded


### multi-features electrodes by roi

In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.metrics import roc_auc_score, accuracy_score
from numpy.random import permutation
from scipy import stats
from matplotlib.offsetbox import AnchoredText
from joblib import delayed, Parallel

st = study('Olfacto')
save_path = path.join(st.path, 'classified/LDA_Power_E_3classes_BBG_no_norm_k5_multi/')
subjects = ['LEFC','MICP','VACJ','SEMC','FERJ','PIRJ','CHAF']
rois = ['ACC','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT']
phase = 'Encoding'
freqs = 4
nperm = 1000
val = 'xpow'

def classif_4_classes_by_subj(su):
    for freq in range(freqs):
        for roi in rois:
            pow_groups, labels, weights, dict_ = groups_pow_scores(phase,su,roi,freq,val)
            freq_name, nelecs = dict_['fname'], dict_['nelecs']
            if nelecs > 0:
                #Filenames to save
                name_auc = (save_path+freq_name+'/auc/'+su +'_auc_3classes_'+roi+'.npy')
                name_perm = (save_path+freq_name+'/auc/'+su +'_perm_3classes_'+roi+'.npy')
                plot_name = (save_path+freq_name+'/fig/'+su +'_Power_3classes_'+roi+'.png')    
                if path.exists(name_auc):
                    print(su,phase,freq,'already computed')
                else:
                    print('--» processing',roi, su, 'freq',freq)

                    pow_data_elec = []
                    for i,power in enumerate(pow_groups):
                        pow_data_elec.append(np.ravel(power))
            # =============================  Classification Computation ============================================================           
                    # create a data matrix, concatenate along the trial dimension
                    x = np.concatenate(pow_data_elec, axis=0)[:,np.newaxis]
                    print ('Size of the concatenated data: ', x.shape)
                    y = np.concatenate([labels]*nelecs,axis=0)
                    print ('Size of label for classif: ', len(y))

                    auc = np.array([])
                    for t in range(x.shape[1]):
                        X = x[:,t]
                        X = X.reshape(-1, 1)
                        score_rep = []
                        for i in range(5):
                            k = 5
                            skf = SKFold(n_splits=k, random_state=None, shuffle=True)
                            skf.get_n_splits(X, y)
                            score_cv = []
                            for train_index, test_index in skf.split(X, y):
                                clf = LDA()
                                X_train, X_test = X[train_index], X[test_index]
                                y_train, y_test = y[train_index], y[test_index]
                                clf.fit(X=X_train, y=y_train)
                                y_pred = clf.predict(X_test)
                                score_cv.append(accuracy_score(y_test,y_pred))
                            score_rep.append(np.mean(score_cv))
                        score_rep = np.asarray(score_rep).reshape(1,len(score_rep))
                        auc = np.vstack((auc, score_rep)) if np.size(auc) else score_rep
                    auc = np.swapaxes(auc,0,1)
                    DA = np.mean(auc)

                    perm_scores = np.array([])
                    for t in range(x.shape[1]):
                        X = x[:,t]
                        X = X.reshape(-1, 1)
                        perm_rep = []
                        for perm in range(nperm):
                            y_perm = y[permutation(len(y))]
                            score_cv = []
                            for train_index, test_index in skf.split(X, y_perm):
                                clf = LDA()
                                X_train, X_test = X[train_index], X[test_index]
                                y_train, y_test = y_perm[train_index], y_perm[test_index]
                                clf.fit(X=X_train, y=y_train)
                                y_pred = clf.predict(X_test)
                                score_cv.append(accuracy_score(y_test,y_pred))
                            perm_rep.append(np.mean(score_cv))
                        perm_rep = np.asarray(perm_rep).reshape(1,len(perm_rep))
                        perm_scores = np.vstack((perm_scores, perm_rep)) if np.size(perm_scores) else perm_rep
                    perm_scores = np.swapaxes(perm_scores,0,1)           
                    th_0_05_perm = perm_pvalue2level(perm_scores, p=0.05, maxst=True)
                    th_0_01_perm = perm_pvalue2level(perm_scores, p=0.01, maxst=True)
                    print('th_perm 005: ', th_0_05_perm[0], '001',th_0_01_perm[0], 
                          'auc_max', np.max(auc), 'auc_mean', np.mean(auc))

            # ============================== PLOT POWER ANALYSIS + STATS & DECODING ACCURACY ===================================================
                    if DA >= th_0_05_perm:
                        # plot and figure parameters
                        xfmt = ScalarFormatter(useMathText=True)
                        xfmt.set_powerlimits((0,3))
                        fig, ax = plt.subplots(figsize=(7,7))
                        title = freq_name+' for '+su+' 4 classes in '+roi
                        fig.suptitle(title, fontsize=12)

                        # Plot the POW + STATS
                        plt.xlabel('EpiScore'), plt.ylabel(freq_name)
                        anchored_text = AnchoredText('DA = %s, th = %s' % (np.round(DA,2), round(th_0_05_perm[0],2)), loc=2)
                        ax.add_artist(anchored_text)
                        xticks, w = np.arange(0,3), 0.8
                        means = [pow.mean()for pow in pow_data_elec]
                        stds = [stats.sem(pow) for pow in pow_data_elec]
                        plt.bar(xticks,means,color='blue',yerr=stds)
                        plt.savefig(plot_name, dpi=300, bbox_inches='tight')
                        plt.clf()
                        plt.close()

                    #Save plots
                    np.save(name_auc, auc)
                    np.save(name_perm, perm_scores)

                    del X, auc, pow_data_elec
            del pow_groups

if __name__ == "__main__":
    Parallel(n_jobs=-1)(delayed(classif_4_classes_by_subj)(su) for su in subjects)

-> Olfacto loaded
